In [24]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup

In [16]:
problems = pd.read_csv('errors_csv3.csv')

In [17]:
errors = problems['type'] == 'e'

In [19]:
problems[errors]

,id,type
1030,129117,e
1151,129351,e
1201,129439,e
1305,129635,e
1376,128663,e
1381,128669,e
1438,128769,e
1477,128843,e
1492,128871,e
1526,128933,e


In [86]:
def get_fighter_matchups(fighter_id):
    page = r.get("http://www.sherdog.com/fighter/index?id={}".format(fighter_id), timeout=3).text
    soup = BeautifulSoup(page, 'html.parser')
    found_divs = soup.find('div', {"class": "module fight_history"})
    for div in found_divs.findAll('div', {'class':'module fight_history'}):
        if (div.find('h2').text == 'Fight History - Pro'):
            table = div
            break
    else:
        raise Exception('Pro fight div not found')
    fighter_name = soup.find('h1', {'itemprop': 'name'}).span.contents[0]
    fighter_table_prep = fighter_id, fighter_name
    cells = []

    event_date = 'NO FIGHTS FOUND'
    return table.findAll('tr')

    for row in table.findAll('tr')[1:]:
        result_cell, fighter_cell, event_cell, *blah = row.findAll('td')
        result = result_cell.span.contents[0]
        opponent_id_url = fighter_cell.contents[0]['href']
        opponent_id = int(opponent_id_url[opponent_id_url.rfind("-") + 1:])
        event_link = event_cell.contents[0]['href']
        event_date = event_cell.findAll('span')[-1].contents[0]
        fighter_out = (fighter_id, opponent_id, result, event_date, event_link)
        cells += [fighter_out]
    return fighter_table_prep, cells, event_date

In [87]:
x = get_fighter_matchups(129351)

Exception: Pro fight div not found

In [81]:
out

<div class="module fight_history">
<div class="module_header">
<h2>Fight History - Pro</h2>
</div>
<div class="content table">
<table border="1">
<tr class="table_head">
<td class="col_one">Result</td>
<td class="col_two">Fighter</td>
<td class="col_three">Event</td>
<td class="col_four">Method/Referee</td>
<td class="col_five">R</td>
<td class="col_six">Time</td>
</tr>
<tr class="odd">
<td><span class="final_result win">win</span></td>
<td><a href="/fighter/Horacio-Gutierrez-172941">Horacio Gutierrez</a></td>
<td><a href="/events/UFC-Fight-Night-78-Magny-vs-Gastelum-42241"><span itemprop="award">UFC Fight Night 78 - Magny vs. Gastelum</span></a><br/><span class="sub_line">Nov / 21 / 2015</span></td>
<td>Decision (Unanimous)<br/><span class="sub_line">Mario Yamasaki</span></td>
<td>3</td>
<td>5:00</td>
</tr>
<tr class="even">
<td><span class="final_result win">win</span></td>
<td><a href="/fighter/Fernando-Perez-33502">Fernando Perez</a></td>
<td><a href="/events/Peru-FC-Peru-Fighting-

In [58]:
a = ['poo']
a[-1]

'poo'

In [90]:
for x in [1,2,3]:
    if x == 2:
        print(x)
        break
else:
    print('ppoooo')

2
